In [16]:
from Crypto.Cipher import AES


def AES_128(key, message):
    a = AES.new(key, AES.MODE_ECB)
    return bytearray(a.encrypt(message))


""" 
The Davies–Meyer single-block-length compression function feeds each block of the message m_{i} as 
the key to a block cipher. 
It feeds the previous hash value H_{i-1} as the plaintext to be encrypted.
The output ciphertext is then also XORed (⊕) with the previous hash value H_{i-1} 
to produce the next hash value H_{i}. 
In the first round when there is no previous hash value it uses a constant pre-specified initial value H_{0}.

(h′)^n (u || v) = Enc(u, v) ⊕ v

Arguments:
    encrypt: an encryption function
    l_key: length in bytes of the keys for encrypt 
    l_message: length in bytes of the messages for encrypt

Returns:
A compression function from messages of length l_key + l_message to messages of length l_message , 
defined by using the Davies -Meyer construction
"""
def davies_meyer(encrypt, l_key, l_message):
    def _compression(message):
        u = message[: l_key]
        v = message[l_key : l_message + l_key]
        h_ = bytearray([a ^ b for a, b in zip(encrypt(u, v), v)])
        
        return h_

    return _compression


"""
In order to make the construction secure, Merkle and Damgård proposed that messages be padded with a padding 
that encodes the length of the original message.
This is called length padding or Merkle–Damgård strengthening.

Arguments:
    message: message to be padded
    l_block: length in bytes of the block
    
Returns:
    extension of message that includes the length of message (in bytes) in its last block
"""
def pad(message, l_block):
    _size = len(message)
    
    if len(message) % l_block != 0:
        merkle_damgard_strength = l_block - (len(message) % l_block) - 1
        message += bytearray([1]) + bytearray(merkle_damgard_strength)
        
    message += (_size % 2 ** (l_block)).to_bytes(l_block, "big")
    
    return message



"""
The Merkle–Damgård hash function builds collision-resistant cryptographic hash functions 
from collision-resistant one-way compression functions.
This construction was used in the design of many popular hash algorithms such as MD5, SHA-1 and SHA-2.

Arguments:
    IV: initialization vector for a hash function
    comp: compression function to be used in the Merkle-Damgard construction
    l_block: length in bytes of the blocks to be used in the Merkle-Damgard construction

Returns:
    A hash function for messages of arbitrary length, defined by using the Merkle -Damgard construction
"""
def merkle_damgard(IV, comp, l_block):
    def _merkle_damgard_hash(clean_data):
        # initialization vector (IV)
        H = IV
        padded_data = pad(clean_data, l_block)
        
        data_size = len(padded_data)

        for i in range(int(data_size / l_block)):
            one_way = padded_data[l_block * i : l_block * (i + 1)] 
            one_way += H
            
            # compression (or compacting) function
            H = comp(one_way)
            
            
        return H
    
    return _merkle_damgard_hash  